# 基于教程7生成基础模型

In [1]:
import openstudio

from lib.baseline_model import BaselineModel

model = BaselineModel()

# make a 2 story, 100m X 50m, 10 zone core/perimeter building
model.add_geometry(length=100, width=50, num_floors=2, floor_to_floor_height=4, plenum_height=1, perimeter_zone_depth=3)

# add windows at a 40% window-to-wall ratio
model.add_windows(wwr=0.4, offset=1, application_type="Above Floor")

#### 主要参考：https://github.com/NREL/OpenStudio-resources/blob/develop/model/simulationtests/pvwatts.py

# 创建光伏板

In [2]:
#首先是几何形状的生成，光伏板其实就是一个矩形平面，假设光伏板为1m*2m的板子
vertices = openstudio.Point3dVector()#矩形的四个顶点，这里是相对参考系坐标，后面需要关联到绝对坐标系
vertices.append(openstudio.Point3d(0, 0, 0))
vertices.append(openstudio.Point3d(1, 0, 0))
vertices.append(openstudio.Point3d(1, 2, 0))
vertices.append(openstudio.Point3d(0, 2, 0))

#然后将板子旋转到需要的角度，当然这一步也可以不要，在上一步直接将坐标设置为绝对坐即可
#Vector3d(1, 0, 0)就是按x轴旋转，(0, 1, 0)按y轴旋转，(0, 0, 1)按z轴旋转，线性代数里的变换矩阵
rotation = openstudio.createRotation(openstudio.Vector3d(1, 0, 0), openstudio.degToRad(30))
vertices = rotation * vertices

#生成遮阳，是的你没看错，就是生成遮阳，光伏板不也是遮阳吗，哈哈
group = openstudio.model.ShadingSurfaceGroup(model)
group.setXOrigin(20)#板子放在哪里，起点的绝对坐标
group.setYOrigin(10)
group.setZOrigin(9)#z应该高于屋顶，案例里面的房子2层高，光伏放在9m高
shade = openstudio.model.ShadingSurface(vertices, model)
shade.setShadingSurfaceGroup(group)

#再将遮阳设置为光伏板
generator = openstudio.model.GeneratorPVWatts(model, shade, 100)#100是板子的功率

#再添加逆变器
inverter = openstudio.model.ElectricLoadCenterInverterPVWatts(model)
electric_load_center_dist = openstudio.model.ElectricLoadCenterDistribution(model)
electric_load_center_dist.addGenerator(generator)
electric_load_center_dist.setInverter(inverter)

True

# 保存模型

In [3]:
model.save_openstudio_osm(osm_save_directory=None, osm_name="model_9.osm")

Saving to C:\sites\git_code\pyep\Python Openstudio Course\model_9.osm
